In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dynamixel import AX12, Arm
import time

In [59]:
PORT = '/dev/cu.usbmodem1101'
BAUD = 1000000

In [60]:
# Test connection to all servos by pinging
# We expect error=0 for all 6 servos.  See https://emanual.robotis.com/docs/en/dxl/protocol1/#error
with AX12(PORT, BAUD) as robot:
    robot.ping(0)
    robot.ping(1)
    robot.ping(2)
    robot.ping(3)
    robot.ping(4)
    robot.ping(5)
    # print('test next msg: ', robot.read_next_msg())
    print('test all msgs: ')
    for msg in robot.read_all_msgs():
        print(msg)

test all msgs: 
Status(id=0x00, error=00000000, data=0)
Status(id=0x01, error=00000000, data=0)
Status(id=0x02, error=00000000, data=0)
Status(id=0x03, error=00000000, data=0)
Status(id=0x04, error=00000000, data=0)
Status(id=0x05, error=00000000, data=0)


In [10]:
# Write speeds of all servos
with Arm(PORT, BAUD) as robot:
    robot.write_all(AX12.MOVING_SPEED, 25, nbytes=2)

In [9]:
# Command base servo to 0 degrees
with Arm(PORT, BAUD) as robot:
    robot.command_angle(0, 0)

In [46]:
# Disable (or enable) all servos
with Arm(PORT, BAUD) as robot:
    robot.write_all(AX12.TORQUE_ENABLE, 0)  # change to 1 to enable

In [47]:
# Command all servo angles
with Arm(PORT, BAUD) as robot:
    robot.command_angles_deg(0, 0, 0, 0, 0)

In [48]:
# Record all joint angles
def joint_angles_string(robot):
    joint_angles = robot.read_all_joint_angles_deg()
    return ' '.join([f'{angle:3.0f}' for angle in joint_angles])
with Arm(PORT, BAUD) as robot:
    while True:
        print(joint_angles_string(robot))
        break
        time.sleep(0.5)

 -0   0  -0  -1  -0  -0


In [61]:
# Paint dipping trajectory
TRAJECTORY_QS_TOLS_TIMEOUTS_PAUSES = [
    ([0, 0, 0, 0, 0], None, None, 0),
    ([0, 60, 120, 0, -90], None, None, 0),
    ([-88, 60, 120, 0, -90], None, None, 0),
    ([-88, 60, 130, 0, -90], 2, 3, 2),
    ([-88, 60, 120, 0, -90], 2, 3, 0),
    ([0, 60, 120, 0, -90], None, None, 0),
    ([0, 60, 60, 0, -90], None, None, 0),
    ([0, 0, 0, 0, 0], None, None, 0),
]

# Follow trajectory
with Arm(PORT, BAUD) as robot:
    robot.write_all(AX12.TORQUE_ENABLE, 1)
    for i, (q, tol, timeout, pause) in enumerate(TRAJECTORY_QS_TOLS_TIMEOUTS_PAUSES):
        print(f'Going to {q} ({i+1}/{len(TRAJECTORY_QS_TOLS_TIMEOUTS_PAUSES)})')
        # robot.go_to_blocking(q, cb=lambda: print(f'{joint_angles_string(robot):80}', end='\r'))
        print_debug = lambda: print(f'          {joint_angles_string(robot):80}', end='\r')
        if robot.go_to_blocking(q, tol=tol, timeout=timeout, cb=print_debug):
            print("SUCCESS!")
        else:
            print("\nTimeout...  Moving on")
        time.sleep(pause)

Going to [0, 0, 0, 0, 0] (1/8)
SUCCESS!
Going to [0, 60, 120, 0, -90] (2/8)
SUCCESS!   -1  61 -61 114  -0 -90                                                         
Going to [-88, 60, 120, 0, -90] (3/8)
SUCCESS!  -82  61 -61 122  -0 -90                                                         
Going to [-88, 60, 130, 0, -90] (4/8)
SUCCESS!  -88  61 -61 128  -0 -90                                                         
Going to [-88, 60, 120, 0, -90] (5/8)
          -88  61 -61 123  -0 -90                                                         
Timeout...  Moving on
Going to [0, 60, 120, 0, -90] (6/8)
SUCCESS!   -6  61 -61 123  -0 -90                                                         
Going to [0, 60, 60, 0, -90] (7/8)
SUCCESS!   -1  61 -61  66  -0 -90                                                         
Going to [0, 0, 0, 0, 0] (8/8)
SUCCESS!   -1   0   0  -0  -0  -6                                                         
